# TLA+
https://www.philipzucker.com/Modelling_TLA_in_z3py/
Alloy
Quintus

https://www.learntla.com/introduction/

Lift everything to signals
https://news.ycombinator.com/item?id=41382828

I need to hide everything under the abstract definitions so that it is readable.
The conversio nin simplify  of implies to no

Ah. This _does_ require induction.

https://lamport.azurewebsites.net/tla/book-02-08-08.pdf specifying systems

There is a tla in isabelle.


In [1]:
from kdrag.all import *
import kdrag.theories.logic.temporal as tla

hr = tla.Int("hr")
HCini = tla.tand(tla.tint(1) <= hr, hr <= tla.tint(12))
HCini = kd.define("HCini", [], HCini)
#kd.rewrite.simp(HCini)
HCnxt = tla.ieq(tla.inext(hr), tla.if_int(tla.ineq(hr, tla.tint(12)), hr + tla.tint(1), tla.tint(1)))
HCnxt = kd.define("HCnxt", [], HCnxt)
kd.rewrite.simp(HCnxt)
#kd.rewrite.simp(tla.Valid(tla.Always(tla.Next(hr) == HCnxt, vs=[hr])))
#HC = tla.tand(HCini, tla.always(HCnxt, vs=[hr]))
HC = tla.tand(HCini, tla.always(HCnxt))
thm = tla.valid(tla.implies(HC, tla.always(HCini)))
thm
#kd.rewrite.simp(thm)




valid(timpl(tand(HCini, always(HCnxt)), always(HCini)))

In [3]:
# intermiedate lemma
l = kd.Lemma(tla.valid(tla.always(tla.implies(tla.tand(HCini, HCnxt),  HCini))))


In [4]:
p, q = smt.Consts("p q", tla.TBool)
valid_and = kd.prove(smt.ForAll([p,q], tla.valid(tla.tand(p, q)) == smt.And(tla.valid(p), tla.valid(q))), by=[tla.valid.defn, tla.tand.defn])
valid_or = kd.prove(smt.ForAll([p,q], tla.valid(tla.tor(p, q)) == smt.Or(tla.valid(p), tla.valid(q))), by=[tla.valid.defn, tla.tor.defn])
valid_impl = kd.prove(smt.ForAll([p,q], tla.valid(tla.timpl(p,q)) == smt.Implies(tla.valid(p), tla.valid(q))), by=[tla.valid.defn, tla.timpl.defn])
valid_not = kd.prove(smt.ForAll([p,q], tla.valid(tla.tnot(p)) == smt.Not(tla.valid(p))), by=[tla.valid.defn, tla.tnot.defn])

tand_0 = kd.prove(smt.ForAll([p], tla.tand(p,q)[0] == smt.And(p[0], q[0])), by=[tla.tand.defn])
tor_0 = kd.prove(smt.ForAll([p], tla.tor(p,q)[0] == smt.Or(p[0], q[0])), by=[tla.tor.defn])

t = smt.Int("t")
tand_t = kd.prove(smt.ForAll([p,q,t], tla.tand(p,q)[t] == smt.And(p[t], q[t])), by=[tla.tand.defn])
tor_t = kd.prove(smt.ForAll([p,q,t], tla.tor(p,q)[t] == smt.Or(p[t], q[t])), by=[tla.tor.defn])
timpl_t = kd.prove(smt.ForAll([p,q,t], tla.timpl(p,q)[t] == smt.Implies(p[t], q[t])), by=[tla.timpl.defn])
tnot_t = kd.prove(smt.ForAll([p,t], tla.tnot(p)[t] == smt.Not(p[t])), by=[tla.tnot.defn])

# one step unfold
always_t = kd.prove(smt.ForAll([p,t], tla.always(p)[t] == smt.And(p[t], tla.always(p)[t+1])), by=[tla.always.defn])


In [7]:
always_next = kd.prove(smt.ForAll([p], tla.valid(tla.always(p)) == smt.And(tla.valid(p), tla.valid(tla.always(tla.bnext(p))))),
                        by=[tla.always.defn, tla.bnext.defn, tla.valid.defn])

LemmaError: ('prove', valid(always(p)) == And(valid(p), valid(always(next(p)))), unknown)

In [36]:
l = kd.Lemma(smt.ForAll([p], tla.valid(tla.always(p)) == smt.And(tla.valid(p), tla.valid(tla.always(tla.bnext(p))))))
p = l.fix()
l.unfold()
l.simp()
l.split()

l.intros()
l.split(-1)
dt = l.fix()
l.instan(1, dt-1)
l.auto()

#l.intros()
l.auto()

always_unfold = l.qed()



|- ForAll(p!6016,
       valid(always(p!6016)) ==
       And(valid(p!6016), valid(always(next(p!6016)))))

In [40]:
kd.prove(smt.ForAll([p], tla.valid(tla.tnot(tla.always(p))) == tla.valid(tla.eventually(tla.tnot(p)))), 
         by = [tla.valid.defn, tla.tnot.defn, tla.always.defn, tla.eventually.defn])

LemmaError: ('prove', valid(tnot(always(p!6033))) ==
valid(eventually(tnot(p!6033))), unknown)

I want it to always refold stuff.
So I should rearrange my definitions to be in terms of...
No these are the extensionally lifted definitions. Hmm.
That's why I kept unfolding and simp.
But simp also mushes around boolean connectives, which I don't love

Yeah, lambda stuff is kind of defined observationally actually.



In [ ]:
def ext_defn(f):
    f.defn

In [3]:

import kdrag.theories.int as int_
l = kd.Lemma(thm)
l.unfold(tla.valid)
l.unfold(tla.timpl, tla.tand)
l.simp()
l.intros()
l.split(0)

l.unfold(tla.always)
l.simp()
dt = l.fix()
#l.intros()
l.induct(dt)
l.auto() # < 0
l.auto() # = 0

n = l.fix()
l
l.intros()
l.split(-1)
l.unfold(tla.always)

[dt!19, n!23];
[val(HCini)[0],
 val(always(HCnxt))[0],
 n!23 >= 0,
 Lambda(dt!19, Or(Not(dt!19 >= 0), val(HCini)[dt!19]))[n!23]]
?|- Lambda(dt!19, Or(Not(dt!19 >= 0), val(HCini)[dt!19]))[n!23 +
  1]

In [ ]:
import kdrag as kd
from kdrag.smt import *

Signal = kd.smt.ArraySort(IntSort(), BoolSort())
n = smt.Const("n", IntSort())
next = kd.define("next", [x], smt.Lambda([x], x[n-1]))

def TConst(name, sort):
    return Const(name, smt.ArraySort(IntSort(), sort))
def TConsts(name, sort):
    return Consts(name, smt.ArraySort(IntSort(), sort))
x,y,z = TConsts("x y z", BoolSort())

def TAnd(*args):
    return Lambda([n], And([arg[n] for arg in args]))

INIT = Const("INIT", Signal)
def BOX(f):
    return Lambda([m], ForAll([n], n >= m, f[n]))
ALWAYS = BOX

BOX = kd.define("[]", [f], BOX(f))

def lower(t):
    return t[0]

lower(TAnd(INIT, Box(next(x) == x)))

LabelSort = EnumSort("LABEL", "L1 L2 L3 L4")
LABEL = TConst("LABEL", LabelSort)


def bounded(n):
    # knuckeldragger could understand that the forall property implies the bounded property
    # And then separaetly to the bounded property.
    return kd.lemma(ForAll[P], Implies(Box(P), And([P(n) for i in range(n)]) ))



hr = TConst("hr", IntSort())
HCini = TAnd(hr >= 1, hr <= 12)
HCnxt = next(hr) == TIf(hr == 12, 1, hr + 1)
HC = TAnd(HCini, Box(HCnxt))


Differentiation and Platzer's stuff.
ODEs


# Intuitionistic Knuckle
Deep embedding vs new kernel


Abstract proof theory

In [ ]:
from kdrag.all import *

# Proofs are kind of open datatype conitaing subproofs and 
Proof = smt.DeclareSort("Proof")
prop = Function("prop", Proof, Prop)
# open datatype? Closed?
Prop = smt.DeclareSort("Prop")
impl = Function("impl", Prop, Prop)
is_impl = Function("is_impl", Prop, BoolSort())


# The semantic interpretation of impl is a function
def Impl(a,b): return ArraySort(Proof, Proof)
# a partial function though.
ArraySort(Proof,Proof,Proof,BoolSort())
ArraySort(Proof,Option(Proof))



wf = Function("wf", Proof, BoolSort())
kd.notation.wf.register(Proof, wf)






In [ ]:
class Proof():
    subproofs : list["Proof"] 
    hyps: list[smt.BoolRef]
    conc: smt.BoolRef

def impE(pfab, pfa):
    pfab.conc 
    hyps = pfab.hyps + [pfab.conc]
    conc = pfa.conc
    return Proof([pfab, pfa], hyps, conc)

def 

def weaken(pfa, newhyp):
    return Proof([pfa], pfa.hyps + [newhyp], pfa.conc)

def swap(pf, i,j):
    return Proof(pf ,pf.hyps, pf.conc)


